In [74]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2
import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

In [75]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text)
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [76]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [77]:
def parse_form(fileloc,title=None,cat=None):
    f = PyPDF2.PdfFileReader(fileloc)
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":fields
            }    
    return stats

In [78]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi],ignore_index=True)
df["pages"] = ""
df["fields"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if row["pages"] == "":
        stats = parse_form("../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["group"])
        df.at[index, 'pages'] = stats["pages"]
        df.at[index, 'fields'] = stats["fields"]
        df.at[index, 'f_per_p'] = stats["avg fields per page"]
        df.at[index, 'reading'] = stats["reading grade level"]
        #print(index)
        
df

,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,f_per_p,reading
0,ed1efd08e3304bdaa887db753509d666,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03,2,[],0.0,10
1,28ab1fb8df5c477cb4353e44f6d5a404,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-03,2,[],0.0,11
2,279083cdac044d22b873a6a6ab8d9a14,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-03,5,[],0.0,15
3,ebad2caa6be0458d9e5c75635a7f66c2,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-03,3,[],0.0,10
4,92932e4f938440ce8b93b6434aedfc3d,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-03,1,[],0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...
75,2ae35d7dfa7e465c99736a9740fd4560,MI,www.courts.michigan.gov,Judgment of Sentence/Commitment to Department ...,juvenile designated forms,https://www.courts.michigan.gov/4a7df7/siteass...,jc72.pdf,2021-11-03,2,[STATE OF MICHIGAN JUDICIAL CIRCUIT FAMILY DI...,52.5,8
76,fd9424f053a44e8280befd0d365c6ca1,MI,www.courts.michigan.gov,Order after Hearing on Petition to Accept Rele...,safe delivery newborn,https://www.courts.michigan.gov/4a2dc0/siteass...,ccfd08.pdf,2021-11-03,1,"[cirno, county, caseno, cname, judge, barno, p...",26.0,26
77,fa8c956052f94a78a0c951328006bd62,MI,www.courts.michigan.gov,Case Evaluator Application,case evaluation,https://www.courts.michigan.gov/4a2af9/siteass...,mc34.pdf,2021-11-03,3,"[raddress, htelno, baddress, btelno, cemployer...",25.333333,22
78,1ceaeb87496540e1b0b46929498536fd,MI,www.courts.michigan.gov,Collecting Your Money from a Small Claims Judg...,small claims,https://www.courts.michigan.gov/4a801f/siteass...,dci84.pdf,2021-11-03,2,[],0.0,9


In [79]:
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [80]:
os.system('cp ../data/processed/form_data.csv docs/forms/form_data.csv')
os.system('cp ../data/raw/www.utcourts.gov/forms/* docs/forms/')
os.system('cp ../data/raw/www.courts.michigan.gov/forms/* docs/forms/')

0